Основная цель этого файла перевод URL ссылок в папки с изображениями.

In [ ]:
import os
import csv
import requests
from tqdm import tqdm
from requests.adapters import HTTPAdapter, Retry
from google.colab import files


## Загрузка csv файлов

In [ ]:
# Загрузка csv файлов
!pip install gdown --quiet
import gdown
file_id = "1585FWwha2MJGWyRvKmLRxmqblvmsjSq3"
url = f"https://drive.google.com/uc?id={file_id}"
output = "george.csv"


file_id1 = "1XjTxMwfcSAZJIlcw2cZzVy1UzUCzIoID"
url1 = f"https://drive.google.com/uc?id={file_id1}"
output1 = 'not_george.csv'

gdown.download(url, output, quiet=False)
gdown.download(url1, output1, quiet=False)

In [ ]:
george_df = pd.read_csv('george.csv', header=None)
not_george_df = pd.read_csv('not_george.csv', header=None)
george_df.head()

## Функция загрузки изображений с URL ссылок

In [ ]:
# Функция загрузки изображений с URL ссылок
def download_images_indexed(csv_path, url_col, out_folder='images'):
    os.makedirs(out_folder, exist_ok=True)

    sess = requests.Session()
    retries = Retry(total=3, backoff_factor=0.3, status_forcelist=[500,502,504])
    sess.mount('http://',  HTTPAdapter(max_retries=retries))
    sess.mount('https://', HTTPAdapter(max_retries=retries))

    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f, fieldnames=[url_col])
        urls = [row[url_col].strip() for row in reader if row[url_col].strip()]

    print(f"Found {len(urls)} URLs in {csv_path} under column '{url_col}'")

    for idx, url in enumerate(tqdm(urls, desc="Downloading"), start=1):
        ext = os.path.splitext(url)[1] or '.jpg'
        fname = f"img_{idx:04d}{ext}"
        path  = os.path.join(out_folder, fname)
        try:
            r = sess.get(url, timeout=10, stream=True)
            r.raise_for_status()
            with open(path, 'wb') as out:
                for chunk in r.iter_content(1024*1024):
                    out.write(chunk)
        except Exception as e:
            print(f"[{idx:04d}] ERROR downloading {url}: {e}")


## Скачивание на локальный компьютер

In [ ]:
download_images_indexed(
  csv_path='george.csv',
  url_col='images_url',
  out_folder='images_george1'
)

In [ ]:
!zip -r images_george1.zip images_george1
files.download('images_george1.zip')


In [ ]:
download_images_indexed(
  csv_path='not_george.csv',
  url_col='image_url',
  out_folder='images_not_george1'
)

In [ ]:
!zip -r images_not_george1.zip images_not_george1
files.download('images_not_george1.zip')
